# Advanced Classes

This tutorial focuses on the invocation of decorators, dunder methods and demystifies the operators in Python.

## Decorators

Consider the following modification of our `Student` class:

In [ ]:
class Student:
    """Stores information about students."""

    def __init__(self, name):
        self.name = name
        self.homework_grades = []
        self.exam_grades = []
        self.extra_credit = None

    @classmethod
    def from_dict(cls, sdict):
        """Initialize a Student using the given dictionary."""
        student = cls(sdict['name'])
        if 'homework_grades' in sdict:
            student.homework_grades = sdict['homework_grades']
        if 'exam_grades' in sdict:
            student.exam_grades = sdict['exam_grades']
        if 'extra_credit' in sdict:
            student.extra_credit = sdict['extra_credit']
        return student

    @staticmethod
    def avg(values):
        return sum(values)/float(len(values))

    @property
    def homework_avg(self):
        """Average homework grade."""
        return self.avg(self.homework_grades)

    @property
    def exam_avg(self):
        """Average exam grade."""
        return self.avg(self.exam_grades)

    @property
    def gpa(self):
        """GPA (on a 4.0 scale)."""
        scores = [self.homework_avg, self.exam_avg]
        if self.extra_credit is not None:
            scores.append(self.extra_credit)
        return 4.0 * self.avg(scores)/100.

What are all of those things with `@` before the function definitions? And why don't the `from_dict` and `avg` methods not have `self` as their first arguments?!?

The names beginning with `@`, such as `@classmethod`, are called *decorators*. Essentially, decorators are wrappers around functions that modify the behavior of that function. They can be used on any Python function, but they are most often seen in classes.

Using and creating decorators is a large topic itself, the details of which we won't get into here (if you're interested, see [here](https://realpython.com/primer-on-python-decorators/) for an excellent tutorial). However, there are three predefined decorators in the Python `builtins` that often come up in class definitions that I want highlight here: `@classmethod`, `@staticmethod`, and `@property`.

### @classmethod

The `@classmethod` decorator modifies methods so that instead of taking an *instance* of a class as the first argument (what we normally call `self`) it takes the *class itself* (which we normally call `cls`). This is typically used to provide a way to instantiate a class using alternate input arguments than what is defined in `__init__`. For example, say we had a dictionary that specifies all of the information about Susie:

In [ ]:
susie_dict = {}
susie_dict['name'] = 'Susie'
susie_dict['homework_grades'] = [80., 73., 77., 50., 0.]
susie_dict['exam_grades'] = [70., 63., 50.]

We can now instantiate a `Student` representation of Susie using the `from_dict` method:

In [ ]:
susie = Student.from_dict(susie_dict)

print(susie.name)
print(susie.homework_grades)
print(susie.exam_grades)

Notice that when we used `from_dict` we only provided the dictionary, even though the definition of `from_dict` had two arguments, `cls` and `sdict`. As with normal methods, `@classmethod` automatically adds in the class as the first argument.

### @staticmethod

The `@staticmethod` decorator modifies methods so that the class instance `self` is *not* automatically added when the method is called. As a result, we do not need to reserve the first argument in the definition of a `@staticmethod`. We see that in the above example: `avg` takes a single argument `values`, which is a list of values to calculate an average for.

Since methods wrapped with `@staticmethod` do not do any automatic substitutions, it's possible to use them without needing to create a class instance. For example:

In [ ]:
Student.avg([60., 70.])

In fact, we could have just defined `avg` outside of `Student` in the global namespace. In that case we would of called `avg(...)` instead of `self.avg(...)` inside of `Student`.

So why use `@staticmethod`? Its main purpose is to provide a function that is heavily used by a class, but may have little meaning outside of the class. This can make code a easier to read and understand ("[syntactic sugar](https://en.wikipedia.org/wiki/Syntactic_sugar)"). Basically, `@staticmethod` is a way of logically organizing functions.

### @property

The `@property` decorator is another form of syntactic sugar that modifies how a method is called. Basically, it makes a method look like an attribute.

For example, in the above `homework_avg`, `exam_avg`, and `gpa` were all methods. Normally you would call them like `susie.gpa()` (as we did in the [Classes](#III.-Classes) section). However, because we stuck the `@property` decorator on each of these methods, we instead do:

In [ ]:
susie.gpa

In other words, we no longer include the `()` after the name. Note that `@property` **only works with methods that take no arguments.**

The `@property` is used when we want to run some additional code under the hood when an attribute is accessed. It is typically paired with a "setter", which allows us to also run some code when an attribute is set. For example:

In [ ]:
class Star:
    _mass = None

    @property
    def mass(self):
        if self._mass is None:
            raise ValueError("no mass set")
        return self._mass

    @mass.setter
    def mass(self, value):
        if value <= 0:
            raise ValueError("mass must be > 0")
        self._mass = value

In [ ]:
star = Star()

In [ ]:
print(star.mass)

In [ ]:
star.mass = 10.
print(star.mass)

In [ ]:
star2 = Star()
star2.mass = -3

## Dunder

Dunder (**D**ouble **under**score) methods `__meth__` are names which are reserved for Python only and should not be invented by us (implemented; yes). Just to be precise, `__meth` is fine.

These methods are used to delegate the actual operator call. When we invoke any operator (`+`, `==`, but also `()` and so on), the object that this is applied on is checked for a corresponding method. For the `+`, the object (on the left) is checked for a `__add__` method. If this is not found or return a `NotImplemented` either alternatives are tried or an error is raised if all possibilities are tried.

Alternatives involve in this case to call the `__radd__` (**r**ight **add**) on the _object on the right_ **IF** the objects are of different types.

### len

The `len` method simply checks if there is a `__len__` implemented.

### str

To have a nice, representable, human readable string, `__str__` should be implemented. There is a similar one, which is `__repr__`. This is also a string representation of the object, yet more targeted towards the developers.

If no `__str__` is provided, it falls back to `__repr__`, which, if not provided, uses a default implementation.

In [ ]:
class Name:
    def __init__(self, name):
        self.name = name
        

class NameRepr(Name):
    def __repr__(self):
        return self.name


class NameStr(Name):
    def __str__(self):
        return f'I am {self.name}'
    

class NameStrRepr(NameStr, NameRepr):
    pass

**Exercise**: try it out by using `str(...)` and `repr(...)`

### Callable

In Python, a callable is any object that can be called. Calling an object means to have `(...)` attached behind it. This operator looks for a `__call__` method.

In [ ]:
class Callable:
    def __call__(self, *args, **kwargs):
        print(f"called with args {args} and kwargs {kwargs}")
        

class NotCallable:
    pass

In [ ]:
call = Callable()
noncall = NotCallable()

In [ ]:
call()

In [ ]:
try:
    noncall()
except TypeError as error:
    print(error)

`TypeError: 'NotCallable' object is not callable` translates to `has no __call__ method`

### Indexing (iterating)

There are a few methods when it comes down to iteration. However, we won't go into these details but rather look at the normal indexing. That is controlled via `__getitem__` and `__setitem__` and invoked with the `[]` operator.

In [ ]:
class Storage:
    def __init__(self, name):
        self.name = name
        self.container = [1, 5, 4]  # just for demonstration
        
    def __getitem__(self, index):
        print(f"getitem of {self.name} invoked with index {index}")
        return self.container[index]
    
    def __setitem__(self, index, item):
        print(f"setitem of {self.name} invoked with index {index} and item {item}")
        self.container[index] = item

In [ ]:
storage = Storage('one')

In [ ]:
storage[2]

In [ ]:
storage[2] = 3

## self

What is actually self? Nothing else than the object itself. However, we can rename it however we like.

*Read the following well*
If an instance is create of a class and a method is called on that instance, the _first_ argument to the method is the instance itself.
**Fullstop**

What are the consequences of this?

In [ ]:
class A:
    def __init__(self, value):
        self.value = value
    
    def add(self, y):
        return self.value + y.value

In [ ]:
a = A(4)
b = A(38)

In [ ]:
a.add(b)

In [ ]:
A.add(a, b)

The latter works as well! Why not? `add` is a method that we call and we give it two arguments. Forgetting about class dynamics, it makes actually complete sense.

## Naming conventions


In Python, there are a number of conventions for naming variables, functions, and classes. These are not enforced by the Python interpreter, but are used to make code more readable and understandable.

1. **Class names** should be in CamelCase (e.g., `MyClass`).
2. **Constant names** should be in all caps with underscores separating words (e.g., `MY_CONSTANT`).
3. Everything else should be in snake_case, that is, all lowercase with underscores separating words (e.g., `my_function`).

For more details, see [PEP 8](https://www.python.org/dev/peps/pep-0008/), the official Python style guide.

Methods of classes have their own naming conventions, some of which have an actual effect on how the method is treated by Python. There are four types:
  - **Public methods** are those that are part of the class's public API. These should be in snake_case, for example, `A.my_method()`.
  - **Private methods** are those that are only to be used within the class. These should begin with an underscore, for example, `_my_private_method()`. This is a convention only; Python does not enforce privacy. These methods should not be used as they can be changed without notice. The use-case to _actually_ use them is usually to "hack" something.
  - **Double private** are methods that begin and end with double underscores. These are not only private but their name is mangled (with a `_class` prepended, where `class` is the actual class name), which is a way of making it difficult to override a method in a subclass. They can, technically, still be called, but it is made purposefully difficult. For example, `__my_double_private_method()` (calling it from the _outside_ will be `_MyClass__my_double_private_method()`).
  -  **Dunder or magic methods** are methods that are called by Python in response to certain events. They always begin *and* end with double underscores. For example, `__init__` is called when an instance of a class is created. Others include `__str__`, `__add__`, `__len__`, etc. For more on these, see [here](https://rszalski.github.io/magicmethods/). They are reserved for Python's internal use and should not be defined by the user.

In [ ]:
class A:
    def plus(self):
        print("plus")

    def _plus(self):
        print("_plus")

    def __plus(self):
        print("__plus")

    def __init__(self):
        print("init")

    def __str__(self):
        return "HELLO"

In [ ]:
a = A()
a.plus()
a._plus()
a._A__plus()  # notice the mangled name! a.__plus() would not work

In [ ]:
class NamedValue:
    def __init__(self, name):
        self.name = name


class ValueLeft(NamedValue):
    def __add__(self, other):
        print(f"add called on {self.name}")
        return 42


class ValueRight(NamedValue):
    def __radd__(self, other):
        print(f"radd called on {self.name}")
        return 24


class Value(ValueRight, ValueLeft):
    pass

**Exercise**: which one can we add and which raise an error? Think or try it out!

In [ ]:
valleft = ValueLeft('val left')
valleft2 = ValueLeft('val left2')

In [ ]:
valleft + valleft2

## Danger zone

The following is only for fun and should _not_ be used in real live, except you do _really_ know what you're doing and at least two independent colleagues agree that this is the right way to go

We have seen that basically everything is an operator and it has a dunder method. Everything? Quiz: what did we miss?

Solution: the `.` the access operator. Yes, you guessed right. Let's override it

First, where are actually all the attributes stored in a class?
Answer: in the `__dict__` attribute.

In [ ]:
a.__dict__

Next quiz: where are the methods (_remark open_) stored?

In [ ]:
a.__class__.__dict__

In [ ]:
A.__dict__

To be clear, there is nothing special about a value attribute and a method: the value attribute happened to be set on the _instance_ while the method happened to be set on the class. But we can have class attributes as well as (not really occuring in reality though) instance methods.

**Disclaimer: the following is EXTREMELY BAD CODING practices and should NEVER be seen in ANY real used code**

In [ ]:
class GetAndSet:
    def __init__(self):
        self.values = [1, 2, 3, 4, 5]
    
    def add(self, y):
        return self.values[0] + y.values[1]
    
    def __getattr__(self, name):
        if name in ('add', 'addition'):
            return self.add
        if name == 'hello':
            print('I am 42')
            
    # we omit the __setattr__, but the game is the same
    

In [ ]:
get = GetAndSet()

In [ ]:
get.add(get)

In [ ]:
get.addition(get)

In [ ]:
get.hello

We can also provoke the same behavior by using the function `getattr` (or `setattr` respectively)

In [ ]:
getattr(get, 'hello')

In [ ]:
get.hi

Quiz: why the above?

Answer: Because the `__getattr__` that we called returns `None` (as any function/method does without an explicit `return`)

In [ ]:
import this

And then there is the maybe most important sentence of all in Python:

**We're all adults here.**

Behave like one when coding ;)